In [ ]:
#!pip install implicit

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
data = pd.read_csv('retail_train.csv')
data.head(2)

In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

In [ ]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [ ]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights=np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False) #используем параметр p метода choice в который передадим веса товаров
    
    
    return recs.tolist()

In [ ]:
#функция для получения весов товаров в зависимости от объёма продаж в денежном эквиваленте
def get_items_weights(df):
    total_sales=df['sales_value'].sum()
    items_weights=df.groupby('item_id').agg({'sales_value':'sum'}).reset_index().rename(columns={'sales_value':'weight'})
    items_weights['weight']=items_weights['weight'].apply(lambda x: x/total_sales)
    return items_weights

In [ ]:
%%time

# your_code

items_weights=get_items_weights(data_train)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

In [ ]:
#!pip install metrics

In [ ]:
# your_code
# Функции из 1-ого вебинара
import os, sys
    
from metrics import precision_at_k, recall_at_k

In [ ]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code

In [ ]:
#функция для получения Топ 5000 товаров по количеству проданного(можно варьировать какой Топ получать и по какому показателю)
def get_top(df, column='quantity', top=5000):
    top_df=df.groupby('item_id').agg({f'{column}':'sum'}).reset_index().sort_values(column, ascending=False).head(5000).item_id.tolist()
    return top_df

In [ ]:
%%time

items = get_top(data_train)

result['top5000_random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

In [ ]:
%%time

items = get_top(data_train, column='sales_value')

result['top5000_sales_random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

In [ ]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

**Видим, что качество случайной рекомендации возросло, но не на столько, как при использовании взвешенной случайной рекомендации**

Посчитаем качество ItemItemRecommender, выбирая число соседей K

In [ ]:
top_5000= get_top(data_train)

In [ ]:
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [ ]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Подбираем количество соседей:

In [ ]:
neighbors=np.arange(1,21)
fltrs=[None,[itemid_to_id[6666]]]
max_score, m_neighbor, m_fltr = 0,0,0
res_dict={'no_fltr':[[],[]],'fltr':[[],[]]}
for neighbor in neighbors:
    for fltr in fltrs:
        current_key='fltr' if fltr else 'no_fltr'
        model = ItemItemRecommender(K=neighbor, num_threads=4)
        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)
        
        result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=fltr, 
                                                recalculate_user=True)
                                                             ])
        score=round(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean(),4)
        res_dict[current_key][0].append(score)
        res_dict[current_key][1].append(neighbor)
        
        if score> max_score:
            max_score, m_neighbor, m_fltr=score, neighbor, 'filtered by 6666' if fltr else 'non filtered'
print(f'Лучший скор: {max_score}, K_neighbors: {m_neighbor}, условие фильтрации: {m_fltr}')

            
        

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(res_dict['fltr'][1], res_dict['fltr'][0], label='filtered by 6666')
plt.plot(res_dict['no_fltr'][1], res_dict['no_fltr'][0], label='non filtered')
plt.xlabel('N-neighbor')
plt.ylabel('precision_at_k')
plt.legend()
plt.show()